# Multi-cycle image registration

It is often advantageous to acquire data in multiple cycles, in which different objects are stained in different imaging cycles. A key step in preprocessing these kinds of data is image registration. This is implemented in ``blimp`` using functions in ``blimp.preprocessing.registration``. This notebook demonstrates the use of these functions for correcting image data.

Note: You must run ``notebooks/0_setup.ipynb`` first to specify the configuration and download the test data.

In [1]:
import napari

from IPython.display import display
import numpy as np
import matplotlib.pyplot as plt
import logging
import skimage

from blimp.data import load_example_data
from blimp.log import configure_logging
from blimp.constants import blimp_config
from blimp.preprocessing.registration import (
    calculate_shifts,
    apply_shifts,
    register_2D,
    transform_2D,
    TransformationParameters
)
from aicsimageio import AICSImage
from pathlib import Path

configure_logging(verbosity=2)
# ensure that example data is downloaded
load_example_data()
# read correct blimp_config -- created with 0_setup.ipynb
blimp_config.config_fname = "blimp.ini"
print(blimp_config)


07-Feb-23 18:31:43 - bfio.backends - WARNING  - Java backend is not available. This could be due to a missing dependency (jpype).


Reading config from /Users/scottberry/source/blimp/notebooks/blimp.ini
unpacked_dir = /Users/scottberry/source/blimp/notebooks/_data/raw
archive_path = /Users/scottberry/source/blimp/notebooks/_data/archive/_data.zip
Reading config from blimp.ini
BLIMPConfig (fname: blimp.ini)
EXPERIMENT_DIR: /Users/scottberry/source/blimp/notebooks/_experiments
BASE_DATA_DIR: /Users/scottberry/source/blimp/notebooks/_data/raw
data_config/exampledata: /Users/scottberry/source/blimp/notebooks/ExampleData_constants.py



Load some images from the examples directory using the ``aicsimageio`` package

In [ ]:
cyc01 = AICSImage(Path(blimp_config.BASE_DATA_DIR) / '_data' / 'operetta_cls_multiplex' / 'cycle_01' / 'r05c03f15-fk1fl1-mip.ome.tiff')
cyc02 = AICSImage(Path(blimp_config.BASE_DATA_DIR) / '_data' / 'operetta_cls_multiplex' / 'cycle_02' / 'r05c03f15-fk1fl1-mip.ome.tiff')

In [ ]:
print(cyc01.channel_names)
print(cyc01.dims)

get some input data

In [ ]:
dapi01 = cyc01.get_image_data("YX",C=0)
dapi02 = cyc02.get_image_data("YX",C=0)

generate a rotated version of dapi02 and crop dapi01 to match size

In [ ]:
dapi02_rotated = skimage.transform.rotate(dapi02,3,preserve_range=True).astype(np.uint16)
# remove outer 100 pixels
dapi02_rotated_crop = dapi02_rotated[100:-100,100:-100]
dapi01_crop = dapi01[100:-100,100:-100]

# multi-cycle data

Data acquired on the operetta in a real two-cycle experiment. 

In [ ]:
viewer0 = napari.view_image(dapi01,colormap="red",blending="additive",name="Cycle 1 - uncorrected")
viewer0.add_image(dapi02,colormap="green",blending="additive",name="Cycle 2 - uncorrected")

In [ ]:
print(register_2D.__doc__)

Align cycle 2 to cycle 1. The ``register_2D`` function uses the [``elastix`` library from ITK](https://github.com/InsightSoftwareConsortium/ITKElastix). More details on specifying transformation parameters is provided in the [Elastix wiki](https://github.com/SuperElastix/elastix/wiki).

In [ ]:
settings = TransformationParameters('rigid')
dapi02_corrected, parameters_cyc01_cyc02 = register_2D(fixed=dapi01, moving=dapi02, settings=settings)

In [ ]:
viewer0.add_image(dapi02_corrected,colormap="blue",blending="additive",name="Cycle 2 - corrected")

## simulated rotations

Try with the simulated data that has been rotated

In [ ]:
viewer1 = napari.view_image(dapi01_crop,colormap="red",blending="additive",name="Cycle 1 - original")
viewer1.add_image(dapi02_rotated_crop,colormap="green",blending="additive",name="Cycle 2 - rotated")

Test first using translation only

In [ ]:
dapi02_rotated_crop_registered, translation_parameters = register_2D(
    fixed=dapi01_crop,
    moving=dapi02_rotated_crop,
    settings=TransformationParameters(transformation_mode='translation')
)

viewer1.add_image(dapi02_rotated_crop_registered,colormap="blue",blending="additive",name="Cycle 1 - registered (translation)")

now allow rotations in addition to translation

In [ ]:
dapi02_rotated_crop_registered_rigid, rigid_parameters = register_2D(
    fixed=dapi01_crop,
    moving=dapi02_rotated_crop,
    settings=TransformationParameters(transformation_mode='rigid')
)

viewer1.add_image(dapi02_rotated_crop_registered_rigid,colormap="blue",blending="additive",name="Cycle 1 - registered (rotation)")

# Saving / loading transformation parameters

To align multiple cycles with one another, we need to apply the transformation settings from the reference channel to the other channels.

In [ ]:
import os
results_path = Path(blimp_config.EXPERIMENT_DIR) / "registration"
if not results_path.exists():
    os.makedirs(results_path)

Save the parameters aligning cycle 1 to cycle 2

In [ ]:
parameters_cyc01_cyc02.save(results_path / "parameters_cyc01_cyc02.txt")

These can be re-loaded using the ``TransformationParameters`` class with ``from_file``

In [ ]:
parameters_cyc01_cyc02_loaded = TransformationParameters(from_file=results_path / 'parameters_cyc01_cyc02.txt')
parameters_cyc01_cyc02_loaded

Now align all channels using these parameters. Note the use of the ``transform_2D`` function, which uses pre-defined parameters.

In [ ]:
cyc02_arrays = [cyc02.get_image_dask_data('YX',C=c) for c in range(cyc02.dims.C)]
cyc02_registered = [transform_2D(moving=arr, parameters=parameters_cyc01_cyc02_loaded) for arr in cyc02_arrays]

and visualize the full multi-channel image

In [ ]:
viewer2 = napari.Viewer()
viewer2.add_image(dapi01,colormap="blue",blending="additive",name="Cycle 1 - channel 0")
viewer2.add_image(cyc01.get_image_dask_data('YX',C=1),colormap="green",blending="additive",name="Cycle 1 - channel 1")
viewer2.add_image(cyc02_registered[0],colormap="blue",blending="additive",name="Cycle 2 - channel 0")
viewer2.add_image(cyc02_registered[1],colormap="green",blending="additive",name="Cycle 2 - channel 1")
viewer2.add_image(cyc02_registered[2],colormap="red",blending="additive",name="Cycle 2 - channel 2")

# Register a list of images

As in the above example, we often want to register a set of images together, using a common channel captured in all images for alignment. This becomes cumbersome using the above framework, but is readily achieved with the higher-level functions, ``calculate_shifts`` and ``apply_shifts``. 

In [ ]:
# defaults to elastix-based registration with 'rigid' transformation
parameters_list = calculate_shifts(
    images=[cyc01,cyc02],
    reference_channel=0,
    reference_cycle=0
)

In [ ]:
print(parameters_list[1])

These can be applied using the ``apply_shifts`` function

In [ ]:
registered_images = apply_shifts([cyc01,cyc02],parameters_list)

Check the results

In [ ]:
viewer3 = napari.Viewer()
for cycle in range(2):
    viewer3.add_image(
        registered_images[cycle].get_image_dask_data('CYX'),
        name=[str(cycle) + '_' + name for name in registered_images[cycle].channel_names],
        channel_axis=0,
        blending='additive')


``apply_shifts`` has a ``crop`` argument, which is ``False`` by default. ``crop=True`` ensures all images are cropped to the same size, equal to the maximum rectangle found in all images.

In [ ]:
registered_images = apply_shifts([cyc01,cyc02],parameters_list,crop=True)
viewer4 = napari.Viewer()
for cycle in range(2):
    viewer4.add_image(
        registered_images[cycle].get_image_dask_data('CYX'),
        name=[str(cycle) + '_' + name for name in registered_images[cycle].channel_names],
        channel_axis=0,
        blending='additive')


# FFT-based registration

An alternative to ``elastix`` is the [``image_registration``](https://github.com/keflavich/image_registration) package, which is intended for image registration where the brightness is “extended” or “spread out” . An interface to this is provided also using ``register_2D_fast``, and ``register_images``/``align_images`` with ``lib='image_registration'``. Functionality is limited to x-y translations and differs from ``elastix`` in that interpolation is not performed. Images are merely shifted by integer values.

In [ ]:
from blimp.preprocessing.registration import register_2D_fast
dapi02_aligned_fast, parameters_fast = register_2D_fast(fixed=dapi01,moving=dapi02)

Results can be viewed with the other alignments

In [ ]:
viewer0.add_image(dapi02_aligned_fast,colormap="green",blending="additive",name="Cycle 2 - corrected (fast)")

In [ ]:
parameters_list = calculate_shifts(
    images=[cyc01,cyc02],
    reference_channel=0,
    reference_cycle=0,
    lib='image_registration'
)

In [ ]:
parameters_list

In [ ]:
registered_images_fast = apply_shifts([cyc01,cyc02],parameters_list,'image_registration')

In [ ]:
viewer5 = napari.Viewer()
for cycle in range(2):
    viewer5.add_image(
        registered_images_fast[cycle].get_image_dask_data('CYX'),
        name=[str(cycle) + '_' + name for name in registered_images_fast[cycle].channel_names],
        channel_axis=0,
        blending='additive')

## Object-oriented interface

The above functionalities are also provided as an object oriented interface, when using the ``BLImage`` class.

In [2]:
from blimp.image import BLImage
cyc01 = BLImage(Path(blimp_config.BASE_DATA_DIR) / '_data' / 'operetta_cls_multiplex' / 'cycle_01' / 'r05c03f15-fk1fl1-mip.ome.tiff')
cyc02 = BLImage(Path(blimp_config.BASE_DATA_DIR) / '_data' / 'operetta_cls_multiplex' / 'cycle_02' / 'r05c03f15-fk1fl1-mip.ome.tiff')

parameters_list = calculate_shifts(
    images=[cyc01,cyc02],
    reference_channel=0,
    reference_cycle=0
)

The default XML parser will be changing from 'xmlschema' to 'lxml' in version 0.4.0.  To silence this warning, please provide the `parser` argument, specifying either 'lxml' (to opt into the new behavior), or'xmlschema' (to retain the old behavior).


After registering the images, transformation parameters can be assigned to the ``BLImage`` object and applied during image loading using ``align=True``

In [8]:
for cycle, image in enumerate([cyc01,cyc02]):
    image.transformation_parameters = parameters_list[cycle]

In [9]:
viewer6 = napari.Viewer()
viewer6.add_image(cyc01.get_image_data("CYX",align=True),channel_axis=0,blending='additive')
viewer6.add_image(cyc02.get_image_data("CYX",align=True),channel_axis=0,blending='additive')

[<Image layer 'Image [2]' at 0x7fa7ac7c9b80>,
 <Image layer 'Image [3]' at 0x7fa798cbcca0>,
 <Image layer 'Image [4]' at 0x7fa75888bfd0>]

The advantage of this approach is that transformation matrices can be pre-calculated and stored for later application, rather than making copies of the data. A similar approach is used for illumination correction.